In [6]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import random
%matplotlib inline

In [2]:
#use GPU
if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
device = torch.device(dev) 

In [4]:
surnames_dataset = open('../data/polish_surnames.txt', 'r').read().splitlines()
surnames = []
for surname in surnames_dataset:
    flag = 0
    surname = surname.lower()
    for s in surname:
        if not s.isalpha() or s in ['x', 'ß', 'á', 'ä', 'é', 'ë','č' ,'š','ö', 'ö', 'ú', 'ü', 'ý', 'ă', 'č', 'ě', 'ľ', 'ő', 'ř', 'š', 'ű'] : 
            flag = 1
            break
    if flag == 0:
            surnames.append(surname)  

['abakanowicz', 'abako', 'abakończyk']


['gimalski', 'trebisz', 'rajczan']

In [49]:
letters = sorted(list(set(''.join(surnames))))
letter_to_int = {l:i+1 for i, l in enumerate(letters)}
letter_to_int['.'] = 0
int_to_letter = {i:l for l, i in letter_to_int.items()}

In [7]:
random.shuffle(surnames) #

In [11]:
a = [1,2,3,4,5,6,7,8,9]
a[1:3],a[3]

([2, 3], 4)

In [56]:
xs, ys = [], []
for s in surnames:
    chars = ['.'] + list(s) + ['.']
    for x,y,z in zip(chars, chars[1:], chars[2:]):
        #turn xy to bigram and z to letter
        iz = letter_to_int[z]
        ix = letter_to_int[x]
        iy = letter_to_int[y]

        xs.append([ix,iy])
        ys.append(iz)

num = xs.nelement()

n_train_len80 = (0.8 * len(xs) / len(xs)).int()# 80%
n_train_len90 = (0.9 * len(xs) / len(xs)).int()# 90%

train_xs = torch.tensor(xs[:n_train_len80]).to(dev)
train_ys = torch.tensor(ys[n_train_len80:n_train_len90]).to(dev)

dev_xs = torch.tensor(xs[n_train_len80:n_train_len90]).to(dev)


#xs[11:14], ys[11:14]

(tensor([[ 1, 16],
         [16, 23],
         [23,  9]]),
 tensor([23,  9, 27]))

In [71]:
x1 = xs[:,0]
x2 = xs[:,1]
#print(x1[11:14], x2[11:14])
x_encoded1 = F.one_hot(x1, num_classes=35).float().to(dev)
x_encoded2 = F.one_hot(x2, num_classes=35).float().to(dev)
#print(x_encoded1[11:14], x_encoded2[11:14])

x_encoded = torch.cat((x_encoded1, x_encoded2), dim=1).to(dev)  
x_encoded[11:14], x_encoded.shape,x_encoded1.shape

(tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 torch.Size([1718817, 70]),
 torch.

In [43]:
# initialize weights
shape = (len(letter_to_int)*2, len(letter_to_int))
#W = torch.randn(shape, requires_grad=True).to(dev)
W = torch.load('trigram-makemore-polish-weights.pt', map_location=torch.device('cpu'))
W.retain_grad()
W.shape

torch.Size([70, 35])

In [45]:
num_of_steps = 2
for k in range(num_of_steps):
    #forward pass 
    logits = x_encoded @ W #predict log counts
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True) #probabilities for next character
    loss = -probs[torch.arange(x_encoded.shape[0]), ys].log().mean() + 0.02*(W**2).mean() 
    print(loss.item())

    #backward pass
    W.grad = None
    loss.backward()

    #update weights
    W.data += -20 * W.grad

2.1932454109191895
2.1932449340820312


In [40]:
#torch.save(W, 'makemore-polish-weights.pt')

In [70]:
#sample from neural net
print("NAZWISKA")
print("========")
for i in range(5):
  out = []
  idxs = [0,0]
  while True:
    x_encoded1 = F.one_hot(torch.tensor([idxs[0]]), num_classes=35).float().to(dev)
    x_encoded2 = F.one_hot(torch.tensor([idxs[1]]), num_classes=35).float().to(dev)

    x_encoded = torch.cat((x_encoded1, x_encoded2), dim=1)  
    logits = x_encoded @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    idxs[0] = idxs[1]
    idxs[1] = torch.multinomial(p, num_samples=1, replacement=True).item()
    if idxs[1] == 0 and len(out) < 4:
      continue
    out.append(int_to_letter[idxs[1]]) 
    if idxs[1] == 0:
      break
  print(''.join(out))

NAZWISKA
ęlewski.
urowiczykirwlecz.
rutaroulusluk.
gerkieski.
eterej.
